In [1]:
!pip install transformers[torch] -q
!pip install numpy matplotlib -q

import warnings
warnings.filterwarnings("ignore")

import torch
import pandas as pd
import numpy as np
import os
from transformers import AutoTokenizer,AutoModelForMaskedLM
from transformers import pipeline
import re

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = AutoModelForMaskedLM.from_pretrained('sampathlonka/San-BERT')

Using cuda device


config.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [2]:
import os
import json
import glob

In [3]:
files=[]
for fi in glob.glob("/kaggle/input/finetune/*.txt"):
    files.append(fi)

In [4]:
files

['/kaggle/input/finetune/3.txt',
 '/kaggle/input/finetune/10.txt',
 '/kaggle/input/finetune/19.txt',
 '/kaggle/input/finetune/5.txt',
 '/kaggle/input/finetune/7.txt',
 '/kaggle/input/finetune/8.txt',
 '/kaggle/input/finetune/18.txt',
 '/kaggle/input/finetune/17.txt',
 '/kaggle/input/finetune/11.txt',
 '/kaggle/input/finetune/0.txt',
 '/kaggle/input/finetune/9.txt',
 '/kaggle/input/finetune/16.txt',
 '/kaggle/input/finetune/1.txt',
 '/kaggle/input/finetune/13.txt',
 '/kaggle/input/finetune/14.txt',
 '/kaggle/input/finetune/12.txt',
 '/kaggle/input/finetune/15.txt',
 '/kaggle/input/finetune/2.txt',
 '/kaggle/input/finetune/4.txt',
 '/kaggle/input/finetune/6.txt']

In [5]:
from datasets import load_dataset
dataset = load_dataset("text", data_files=files)

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-46ee42a2ecad3455/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
print(len(dataset["train"]))

29310


In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 29310
    })
})

In [8]:
from transformers import BertTokenizerFast

new_model_max_length = 512

# Instantiate the tokenizer
tokenizer = BertTokenizerFast.from_pretrained('sampathlonka/San-BERT',model_max_length = new_model_max_length)

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/472k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/951k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [9]:
tokenizer

BertTokenizerFast(name_or_path='sampathlonka/San-BERT', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, return_tensors="pt")

In [11]:
from transformers import TrainingArguments
from transformers import Trainer

In [12]:
flist=[]
flist=dataset["train"]['text']

In [13]:
len(flist)

29310

In [14]:
def tokenize_function(examples):
    return tokenizer(examples["text"], max_length=64, truncation=True, padding="max_length")

In [15]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=2, remove_columns=["text"])

#0:   0%|          | 0/15 [00:00<?, ?ba/s]

#1:   0%|          | 0/15 [00:00<?, ?ba/s]

In [16]:
def copy_input_ids(example):
    example["labels"] = example["input_ids"].copy()
    return example

In [17]:
tokenized_datasets = tokenized_datasets.map(copy_input_ids)

  0%|          | 0/29310 [00:00<?, ?ex/s]

In [18]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 29310
    })
})

In [19]:
training_args = TrainingArguments(
    output_dir="San-Bert-Final",
    learning_rate=3e-3,
    num_train_epochs=1,
    weight_decay=0.001,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,9.367600
1000,8.807200
1500,8.756500
2000,8.598400
2500,8.616100
3000,8.599400
3500,8.546000


TrainOutput(global_step=3664, training_loss=8.747238558973287, metrics={'train_runtime': 389.734, 'train_samples_per_second': 75.205, 'train_steps_per_second': 9.401, 'total_flos': 964316632896000.0, 'train_loss': 8.747238558973287, 'epoch': 1.0})

In [20]:
from huggingface_hub import notebook_login
notebook_login()

In [21]:
trainer.push_to_hub()

ValueError: Token is required (write-access action) but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.